In [ ]:
#setup - one-time tasks (run from terminal, not jupyter)
pip install cca-zoo

In [1]:
#import libraries
import pandas as pd
import numpy as np
import scipy
import os

userhome = os.path.expanduser('~')

In [33]:
# load residualized and scaled clinical features (from R script)
#stricter exclusion criteria
#clin_file = userhome + r'/Desktop/OCS_bigdata_ML_local/data/networklevel-stricterexclusion/clin_features_scaled_discovery.csv'
#larger sample
clin_file = userhome + r'/Desktop/OCS_bigdata_ML_local/data/networklevel/clin_features_scaled_discovery.csv'

clin_features = pd.read_csv(clin_file)
clin_features = clin_features.to_numpy()
print(type(clin_features))
print(clin_features.shape)

<class 'numpy.ndarray'>
(4701, 8)


In [35]:
# load residualized and scaled imaging features (from R script)
#stricter exclusion criteria
#rsFC_file = userhome + r'/Desktop/OCS_bigdata_ML_local/data/networklevel-stricterexclusion/rsFC_features_scaled_discovery.csv'
#larger sample
rsFC_file = userhome + r'/Desktop/OCS_bigdata_ML_local/data/networklevel/rsFC_features_scaled_discovery.csv'

rsFC_feat_chunks = pd.read_csv(rsFC_file, chunksize=1000)
rsFC_features = pd.concat(rsFC_feat_chunks)
#rsFC_features.sample(10)
rsFC_features = rsFC_features.to_numpy()
print(type(rsFC_features))
print(rsFC_features.shape)

<class 'numpy.ndarray'>
(4701, 91)


In [36]:
# Setup hyperparameter optimization: initialize arrays to store from each CV loop
eCCA_eachfold_c1 = np.empty((10,27), dtype=object) 
eCCA_eachfold_c2 = np.empty((10,27), dtype=object) 
eCCA_eachfold_l1_ratio = np.empty((10,27), dtype=object)
eCCA_eachfold_score_TRAIN = np.empty((10,27), dtype=object) 
eCCA_eachfold_score_TEST = np.empty((10,27), dtype=object) 

print(eCCA_eachfold_c1)

[[None None None None None None None None None None None None None None
  None None None None None None None None None None None None None]
 [None None None None None None None None None None None None None None
  None None None None None None None None None None None None None]
 [None None None None None None None None None None None None None None
  None None None None None None None None None None None None None]
 [None None None None None None None None None None None None None None
  None None None None None None None None None None None None None]
 [None None None None None None None None None None None None None None
  None None None None None None None None None None None None None]
 [None None None None None None None None None None None None None None
  None None None None None None None None None None None None None]
 [None None None None None None None None None None None None None None
  None None None None None None None None None None None None None]
 [None None None Non

In [38]:
#hyperparam optimization
from cca_zoo.models import ElasticCCA

eCCA_model = ElasticCCA(max_iter=100000)
eCCA_param_grid = {"c" : [[1e-2,1e-1,2e-1], [1e-2,1e-1,2e-1]], "l1_ratio" : [0.5,0.67,0.75]}
# c is lasso alpha for each view
# l1_ratio is l1 ratio in lasso subproblems
# higher l1_ratio = more sparse

for foldind in range(0,10):
    #foldind = 0 #remember python starts indexing at 0, while R and Matlab start at 1

    # load 10-fold cross validation (CV) splits
    #thisfold_file = userhome + r'/Desktop/OCS_bigdata_ML_local/data/networklevel-stricterexclusion/partitions2_rep01fold' + str(foldind + 1) + r'.csv'
    thisfold_file = userhome + r'/Desktop/OCS_bigdata_ML_local/data/networklevel/partitions2_rep01fold0' + str(foldind + 1) + r'.csv'
    print(thisfold_file)
    thisfold_file = pd.read_csv(thisfold_file)
    thisfold_file = thisfold_file.to_numpy()
    print(type(thisfold_file))
    print(thisfold_file.shape)

    # for each CV fold, extract only the selected participants and features
    # NB: put this in loop later once troubleshooting complete
    # NB: row and col indices were generated in R/Matlab, which index from 1. python indexes from 0
    thisfold_clin_features = clin_features[(thisfold_file - 1), :]
    #print(thisfold_clin_features.shape)
    thisfold_clin_features = np.squeeze(thisfold_clin_features)
    print(thisfold_clin_features.shape)
    #print(thisfold_clin_features)

    thisfold_rsFC_features = rsFC_features[(thisfold_file - 1), :]
    thisfold_rsFC_features = np.squeeze(thisfold_rsFC_features)
    print(thisfold_rsFC_features.shape)

    #thisfoldfeats = CVfold_features[foldind, :][~np.isnan(CVfold_features[foldind, :])]
    #thisfold_rsFC_features = rsFC_features[(thisfold_file - 1), thisfoldfeats.astype(int)-1]
    #print(thisfold_rsFC_features.shape)
    #print(thisfold_rsFC_features)


    #test set for this CV fold
    thisfold_clin_features_TEST = np.delete(clin_features,thisfold_file-1, axis=0)
    print(thisfold_clin_features_TEST.shape)
    thisfold_rsFC_features_TEST = np.delete(rsFC_features,thisfold_file-1, axis=0)
    thisfold_rsFC_features_TEST = np.squeeze(thisfold_rsFC_features_TEST)
    #thisfold_rsFC_features_TEST = thisfold_rsFC_features_TEST[:, thisfoldfeats.astype(int) - 1]
    print(thisfold_rsFC_features_TEST.shape)
    
    paramind = 0
    for thisc1 in eCCA_param_grid["c"][0]:
        for thisc2 in eCCA_param_grid["c"][1]:
            for thisl1_ratio in eCCA_param_grid["l1_ratio"]:
                eCCA_fit = ElasticCCA(c = (thisc1, thisc2), l1_ratio = thisl1_ratio, latent_dims=1).fit([thisfold_clin_features, thisfold_rsFC_features])
                eCCA_eachfold_c1[foldind,paramind] = thisc1
                eCCA_eachfold_c2[foldind,paramind] = thisc2
                eCCA_eachfold_l1_ratio[foldind,paramind] = thisl1_ratio
                print('params: ' + str(thisc1) + ', ' + str(thisc2) + ', ' + str(thisl1_ratio))
                eCCA_eachfold_score_TRAIN[foldind,paramind] = eCCA_fit.score((thisfold_clin_features, thisfold_rsFC_features))[0]
                print('CCA score (train): ' + str(eCCA_eachfold_score_TRAIN[foldind,paramind]))
                eCCA_eachfold_score_TEST[foldind,paramind] = eCCA_fit.score((thisfold_clin_features_TEST, thisfold_rsFC_features_TEST))[0]
                print('CCA score (test): ' + str(eCCA_eachfold_score_TEST[foldind,paramind]))

                paramind = paramind + 1

/Users/marshlab2/Desktop/OCS_bigdata_ML_local/data/networklevel/partitions2_rep01fold01.csv
<class 'numpy.ndarray'>
(4232, 1)
(4232, 8)
(4232, 91)
(469, 8)
(469, 91)


/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 0 not converged. Increase number of iterations.
  warnings.warn(


params: 0.01, 0.01, 0.5
CCA score (train): 0.19212314340498193
CCA score (test): 0.04031665293289777


/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 0 not converged. Increase number of iterations.
  warnings.warn(


params: 0.01, 0.01, 0.67
CCA score (train): 0.18759639258544314
CCA score (test): 0.03337778367632804


...

/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 0 not converged. Increase number of iterations.
  warnings.warn(


params: 0.2, 0.2, 0.5
CCA score (train): 0.05609905382227076
CCA score (test): 0.06646486388627948


/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 0 not converged. Increase number of iterations.
  warnings.warn(


params: 0.2, 0.2, 0.67
CCA score (train): 0.04708770090079328
CCA score (test): 0.044312044083924196
params: 0.2, 0.2, 0.75
CCA score (train): 0.047087700900793505
CCA score (test): 0.044312044083924196


/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 0 not converged. Increase number of iterations.
  warnings.warn(


In [43]:
#scratch - check that output is working
eCCA_eachfold_score_TRAIN

array([[0.19212314340498193, 0.18759639258544314, 0.18559664080371618,
        0.12072997747625824, 0.10771199533559361, 0.09687910646673026,
        0.08432601382596538, nan, nan, 0.18021126668789322,
        0.17462113237319965, 0.17084610705322079, 0.10296380644509462,
        0.07562445996384204, 0.0710795813519034, 0.07628268978482478,
        nan, nan, 0.17490222476848682, 0.15506943890017721,
        0.15033878452007698, 0.06190666217740226, nan, nan,
        0.07238720691384692, 0.0497777082299975, nan],
       [0.19098669428167225, 0.18640740660051658, 0.1839020372338085,
        0.1087310305421878, 0.09650152921766786, 0.09305958554645755,
        0.08768777608238443, 0.0787504408687465, 0.0751405639324183,
        0.1764596531622873, 0.16931515884810233, 0.16152617215550436,
        0.08964486442095354, 0.07702106998524205, 0.07700487366287079,
        0.07959695389323196, 0.07699868737768334, 0.06985442387698804,
        0.17099162608619878, 0.14666142214977262, 0.143256767

In [46]:
#average TRAIN and TEST correlations over CV folds

#print(eCCA_eachfold_score_TRAIN[0:4,:])
#eCCA_average_score_TRAIN = np.nanmean(eCCA_eachfold_score_TRAIN[:,:], axis=0)
#print(eCCA_average_score_TRAIN)

#print('\n')

#eCCA_average_score_TEST = np.nanmean(eCCA_eachfold_score_TEST[:,:], axis=0)
#print(eCCA_average_score_TEST)
#print(max(eCCA_average_score_TEST))

#print('\n')
#print(eCCA_eachfold_c1)
#print('\n')
#print(eCCA_eachfold_c2)
#print('\n')
#print(eCCA_eachfold_l1_ratio)

# save files
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/stricterexclusion/eCCA_eachfold_c1.csv', eCCA_eachfold_c1, delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/stricterexclusion/eCCA_eachfold_c2.csv', eCCA_eachfold_c2, delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/stricterexclusion/eCCA_eachfold_l1_ratio.csv', eCCA_eachfold_l1_ratio, delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/stricterexclusion/eCCA_eachfold_score_TRAIN.csv', eCCA_eachfold_score_TRAIN, delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/stricterexclusion/eCCA_eachfold_score_TEST.csv', eCCA_eachfold_score_TEST, delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/eCCA_eachfold_c1.csv', eCCA_eachfold_c1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/eCCA_eachfold_c2.csv', eCCA_eachfold_c2, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/eCCA_eachfold_l1_ratio.csv', eCCA_eachfold_l1_ratio, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/eCCA_eachfold_score_TRAIN.csv', eCCA_eachfold_score_TRAIN, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/eCCA_eachfold_score_TEST.csv', eCCA_eachfold_score_TEST, delimiter=',')

# END OF HYPERPARAMETER OPTIMIZATION CODE


In [47]:
#Run elastic net CCA using full discovery sample and optimized hyperparameters
from cca_zoo.models import ElasticCCA
eCCA_model = ElasticCCA(max_iter=100000000000)

#optimized for network level with stricter exclusions
#optim_c1 = 0.01
#optim_c2 = 0.01
#optim_l1_ratio = 0.5

#optimized for network level with larger sample (less strict exclusions)
optim_c1 = 0.01
optim_c2 = 0.01
optim_l1_ratio = 0.5

dims = min(np.linalg.matrix_rank(rsFC_features),np.linalg.matrix_rank(clin_features))
print('dims: ' + str(dims))

eCCA_fit = ElasticCCA(c = (optim_c1, optim_c2), l1_ratio = optim_l1_ratio, latent_dims=dims).fit([clin_features, rsFC_features])
print('params: ' + str(optim_c1) + ', ' + str(optim_c2) + ', ' + str(optim_l1_ratio))
allsubsample_scores = eCCA_fit.score((clin_features, rsFC_features))
print('All discovery CCA scores: ' + str(allsubsample_scores))

#calculate transformed canonical variate values
U,V = eCCA_fit.transform([clin_features, rsFC_features])



dims: 8


/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 0 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 1 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 2 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 3 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 4 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 5 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaco

params: 0.01, 0.01, 0.5
All discovery CCA scores: [0.18398392 0.12487057 0.13751847 0.1298092  0.11734085 0.10230966
 0.10803429 0.0913065 ]


/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 7 not converged. Increase number of iterations.
  warnings.warn(


In [48]:
# Apply fitted enCCA model to test subset data
# load residualized and scaled clinical features (from R script)
#clin_file_test = userhome + r'/Desktop/OCS_bigdata_ML_local/data/networklevel-stricterexclusion/clin_features_scaled_test.csv'
clin_file_test = userhome + r'/Desktop/OCS_bigdata_ML_local/data/networklevel/clin_features_scaled_test.csv'
clin_features_test = pd.read_csv(clin_file_test)
clin_features_test = clin_features_test.to_numpy()
print(type(clin_features_test))
print(clin_features_test.shape)

# load residualized and scaled imaging features (from R script)
#rsFC_file_test = userhome + r'/Desktop/OCS_bigdata_ML_local/data/networklevel-stricterexclusion/rsFC_features_scaled_test.csv'
rsFC_file_test = userhome + r'/Desktop/OCS_bigdata_ML_local/data/networklevel/rsFC_features_scaled_test.csv'
rsFC_feat_chunks = pd.read_csv(rsFC_file_test, chunksize=1000)
rsFC_features_test = pd.concat(rsFC_feat_chunks)
#rsFC_features.sample(10)
rsFC_features_test = rsFC_features_test.to_numpy()
print(type(rsFC_features_test))
print(rsFC_features_test.shape)

# extract discovery-selected features
#allsubsample_rsFC_features_test = rsFC_features_test[:, allsubsample_selected_features[:,0] - 1]
#print(allsubsample_rsFC_features_test.shape)

#calculate transformed canonical variate values
U_test,V_test = eCCA_fit.transform([clin_features_test, rsFC_features_test])

print(U_test[:,0].shape)
print(V_test.shape)

allsubsample_scores_test = eCCA_fit.score((clin_features_test, rsFC_features_test))
print('All test CCA scores: ' + str(allsubsample_scores_test))

from scipy.stats.stats import pearsonr
#calculate correlation coefficient and p-value between x and y
print(pearsonr(U_test[:,0], V_test[:,0]))
print(pearsonr(U_test[:,1], V_test[:,1]))
print(pearsonr(U_test[:,2], V_test[:,2]))

<class 'numpy.ndarray'>
(237, 8)
<class 'numpy.ndarray'>
(237, 91)
(237,)
(237, 8)
All test CCA scores: [ 0.05296232  0.04505123 -0.01652437  0.00999201 -0.00853706 -0.10864947
  0.08786295  0.00512762]
(0.05296231715406499, 0.4170191842133072)
(0.045051227291949617, 0.4900442242931696)
(-0.0165243700391611, 0.8002205040017882)


In [49]:
# Apply fitted enCCA model to replication subset data
# load residualized and scaled clinical features (from R script)
#clin_file_replication = userhome + r'/Desktop/OCS_bigdata_ML_local/data/networklevel-stricterexclusion/clin_features_scaled_replication.csv'
clin_file_replication = userhome + r'/Desktop/OCS_bigdata_ML_local/data/networklevel/clin_features_scaled_replication.csv'
clin_features_replication = pd.read_csv(clin_file_replication)
clin_features_replication = clin_features_replication.to_numpy()
print(type(clin_features_replication))
print(clin_features_replication.shape)

# load residualized and scaled imaging features (from R script)
#rsFC_file_replication = userhome + r'/Desktop/OCS_bigdata_ML_local/data/networklevel-stricterexclusion/rsFC_features_scaled_replication.csv'
rsFC_file_replication = userhome + r'/Desktop/OCS_bigdata_ML_local/data/networklevel/rsFC_features_scaled_replication.csv'
rsFC_feat_chunks = pd.read_csv(rsFC_file_replication, chunksize=1000)
rsFC_features_replication = pd.concat(rsFC_feat_chunks)
rsFC_features_replication = rsFC_features_replication.to_numpy()
print(type(rsFC_features_replication))
print(rsFC_features_replication.shape)

# extract discovery-selected features
#allsubsample_rsFC_features_replication = rsFC_features_replication[:, allsubsample_selected_features[:,0] - 1]
#print(allsubsample_rsFC_features_replication.shape)

#calculate transformed canonical variate values
U_replication,V_replication = eCCA_fit.transform([clin_features_replication, rsFC_features_replication])
#print(U_test[:,0].shape)
#print(V_test.shape)

allsubsample_scores_replication = eCCA_fit.score((clin_features_replication, rsFC_features_replication))
print('All replication CCA scores: ' + str(allsubsample_scores_replication))

#calculate correlation coefficient and p-value between x and y
print(pearsonr(U_replication[:,0], V_replication[:,0]))
print(pearsonr(U_replication[:,1], V_replication[:,1]))
print(pearsonr(U_replication[:,2], V_replication[:,2]))

<class 'numpy.ndarray'>
(4657, 8)
<class 'numpy.ndarray'>
(4657, 91)
All replication CCA scores: [ 0.03247516  0.01257676 -0.01559842  0.05258385 -0.00184925 -0.00390338
  0.01819992 -0.02154381]
(0.032475157672906374, 0.026680115496863)
(0.012576763437265983, 0.39085395632178266)
(-0.015598423247342475, 0.2872154837564367)


In [50]:
# View clinical loadings
#alldiscovery_loadings = eCCA_fit.get_loadings((clin_features, alldiscovery_rsFC_features))
allsubsample_loadings = eCCA_fit.get_loadings((clin_features, rsFC_features), normalize=True)
allsubsample_loadings_u = allsubsample_loadings[0]
allsubsample_loadings_v = allsubsample_loadings[1]

print(allsubsample_loadings_u.shape)
print(allsubsample_loadings_v.shape)

print('All subsample - clinical loadings CV1: \n' + str(allsubsample_loadings_u[:,0]))
print('All subsample - clinical loadings CV2: \n' + str(allsubsample_loadings_u[:,1]))
print('All subsample - clinical loadings CV3: \n' + str(allsubsample_loadings_u[:,2]))
print('All subsample - clinical loadings CV4: \n' + str(allsubsample_loadings_u[:,3]))
print('All subsample - clinical loadings CV5: \n' + str(allsubsample_loadings_u[:,4]))
print('All subsample - clinical loadings CV6: \n' + str(allsubsample_loadings_u[:,5]))
print('All subsample - clinical loadings CV7: \n' + str(allsubsample_loadings_u[:,6]))
print('All subsample - clinical loadings CV8: \n' + str(allsubsample_loadings_u[:,7]))



(8, 8)
(91, 8)
All subsample - clinical loadings CV1: 
[-34.63470187 -32.00574605   6.03433404 -18.30215467 -10.7360363
 -29.46915878 -28.59449344 -17.22629356]
All subsample - clinical loadings CV2: 
[-14.49249544   1.85421846  22.279922     5.2418144  -42.00233535
 -36.17214514 -35.26076719  -1.22295597]
All subsample - clinical loadings CV3: 
[ -5.25402076   1.66106917   1.00250293   2.27415736   1.42933895
 -55.09337468 -19.11640748   3.58222274]
All subsample - clinical loadings CV4: 
[  0.65124947  -5.47067924  -2.59761416  -6.60150965   1.8355035
   8.38205067  30.27357486 -39.54430053]
All subsample - clinical loadings CV5: 
[-33.34232605 -25.4187601  -49.52519299 -45.05326706 -14.71778005
 -13.0646994  -14.40086214 -29.27564986]
All subsample - clinical loadings CV6: 
[ -9.12977212  10.88810218  -4.49362245  -3.64598743   5.44093545
 -17.68232253 -54.13733554 -15.64375093]
All subsample - clinical loadings CV7: 
[  1.13320196 -16.22197785 -17.4152203   37.69996624   0.95492119

In [51]:
# Save clin loadings
#discovery
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_clinloadings_u_CV1norm.csv', allsubsample_loadings_u[:,0], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_clinloadings_u_CV2norm.csv', allsubsample_loadings_u[:,1], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_clinloadings_u_CV3norm.csv', allsubsample_loadings_u[:,2], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_clinloadings_u_CV4norm.csv', allsubsample_loadings_u[:,3], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_clinloadings_u_CV5norm.csv', allsubsample_loadings_u[:,4], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_clinloadings_u_CV6norm.csv', allsubsample_loadings_u[:,5], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_clinloadings_u_CV7norm.csv', allsubsample_loadings_u[:,6], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_clinloadings_u_CV8norm.csv', allsubsample_loadings_u[:,7], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_clinloadings_allnorm.csv', allsubsample_loadings_u, delimiter=',')

#replication
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_clinloadings_u_CV1norm.csv', allsubsample_loadings_u[:,0], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_clinloadings_u_CV2norm.csv', allsubsample_loadings_u[:,1], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_clinloadings_u_CV3norm.csv', allsubsample_loadings_u[:,2], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_clinloadings_u_CV4norm.csv', allsubsample_loadings_u[:,3], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_clinloadings_u_CV5norm.csv', allsubsample_loadings_u[:,4], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_clinloadings_u_CV6norm.csv', allsubsample_loadings_u[:,5], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_clinloadings_u_CV7norm.csv', allsubsample_loadings_u[:,6], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_clinloadings_u_CV8norm.csv', allsubsample_loadings_u[:,7], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_clinloadings_allnorm.csv', allsubsample_loadings_u, delimiter=',')


In [52]:
# View rs-FC loadings
allsubsample_loadings_v_CV1 = np.argsort(abs(allsubsample_loadings_v[:,0]))
#print(alldiscovery_loadings_v_CV1.shape)
print('CV1')
print(allsubsample_loadings_v_CV1[-20:] + 1)
print(allsubsample_loadings_v[allsubsample_loadings_v_CV1[-20:],0])

allsubsample_loadings_v_CV2 = np.argsort(abs(allsubsample_loadings_v[:,1]))
print('CV2')
print(allsubsample_loadings_v_CV2[-20:] + 1)
print(allsubsample_loadings_v[allsubsample_loadings_v_CV2[-20:],1])

allsubsample_loadings_v_CV3 = np.argsort(abs(allsubsample_loadings_v[:,2]))
print('CV3')
print(allsubsample_loadings_v_CV3[-20:] + 1)
print(allsubsample_loadings_v[allsubsample_loadings_v_CV3[-20:],2])

allsubsample_loadings_v_CV4 = np.argsort(abs(allsubsample_loadings_v[:,3]))
print('CV4')
print(allsubsample_loadings_v_CV4[-20:] + 1)
print(allsubsample_loadings_v[allsubsample_loadings_v_CV4[-20:],3])

allsubsample_loadings_v_CV5 = np.argsort(abs(allsubsample_loadings_v[:,4]))
allsubsample_loadings_v_CV6 = np.argsort(abs(allsubsample_loadings_v[:,5]))
allsubsample_loadings_v_CV7 = np.argsort(abs(allsubsample_loadings_v[:,6]))
allsubsample_loadings_v_CV8 = np.argsort(abs(allsubsample_loadings_v[:,7]))

CV1
[89 45 54 61 52 17 49 88 47 25 39 24 19 63 58 37 18 62 16 38]
[ -7.37856194   7.61300117  -8.51610414  -8.95747791  -8.95768572
   9.17853975  -9.59029479   9.60170481  11.53026753  12.1432499
 -12.25267612 -12.77633177 -13.68890766  14.95454849  15.0655032
  15.5099868  -17.61867787 -18.29073372 -20.32654534 -22.66317896]
CV2
[40 45 89 75 22 61 78 14 13 91 11  6 59 74 90 23 63 83 58 77]
[ 12.02641269  12.3915588   12.78328319 -12.96695738  12.99743015
  13.66067346 -13.75446398  14.06720274 -14.15387891  14.44749154
  14.63213658  14.74131201  15.03380643 -15.31957132 -15.5376584
  15.83883164 -16.2285018   16.24790635 -16.9343013  -20.71555081]
CV3
[ 8 84 10 74 79 61 18 22 58 73 71  1 66 14 56 43 19 20  7 67]
[  9.1066386   -9.75198843  -9.87077634  10.00458209  10.09239593
 -10.35739368 -10.53798897 -11.24173151  11.24905889  11.37268581
 -11.64415031 -11.65981607  12.524155   -12.62754448 -12.63927694
  12.86339439  13.13769581  13.42876072  13.877894    18.89520778]
CV4
[51  1

In [53]:
# Save rsFC loadings
#discovery
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_rsFCinds_v_CV1.csv', allsubsample_loadings_v_CV1 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_rsFCloadings_v_CV1.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV1,0], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_rsFCinds_v_CV2.csv', allsubsample_loadings_v_CV2 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_rsFCloadings_v_CV2.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV2,1], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_rsFCinds_v_CV3.csv', allsubsample_loadings_v_CV3 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_rsFCloadings_v_CV3.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV3,2], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_rsFCinds_v_CV4.csv', allsubsample_loadings_v_CV4 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_rsFCloadings_v_CV4.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV4,3], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_rsFCinds_v_CV5.csv', allsubsample_loadings_v_CV5 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_rsFCloadings_v_CV5.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV5,4], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_rsFCinds_v_CV6.csv', allsubsample_loadings_v_CV6 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_rsFCloadings_v_CV6.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV6,5], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_rsFCinds_v_CV7.csv', allsubsample_loadings_v_CV7 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_rsFCloadings_v_CV7.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV7,6], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_rsFCinds_v_CV8.csv', allsubsample_loadings_v_CV8 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/alldiscovery_rsFCloadings_v_CV8.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV8,7], delimiter=',')

#replication
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCinds_v_CV1.csv', allsubsample_loadings_v_CV1 + 1, delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCloadings_v_CV1.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV1,0], delimiter=',')

#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCinds_v_CV2.csv', allsubsample_loadings_v_CV2 + 1, delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCloadings_v_CV2.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV2,1], delimiter=',')

#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCinds_v_CV3.csv', allsubsample_loadings_v_CV3 + 1, delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCloadings_v_CV3.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV3,2], delimiter=',')

#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCinds_v_CV4.csv', allsubsample_loadings_v_CV4 + 1, delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCloadings_v_CV4.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV4,3], delimiter=',')

#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCinds_v_CV5.csv', allsubsample_loadings_v_CV5 + 1, delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCloadings_v_CV5.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV5,4], delimiter=',')

#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCinds_v_CV6.csv', allsubsample_loadings_v_CV6 + 1, delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCloadings_v_CV6.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV6,5], delimiter=',')

#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCinds_v_CV7.csv', allsubsample_loadings_v_CV7 + 1, delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCloadings_v_CV7.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV7,6], delimiter=',')

#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCinds_v_CV8.csv', allsubsample_loadings_v_CV8 + 1, delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCloadings_v_CV8.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV8,7], delimiter=',')

In [55]:
# Save canonical variates for all subsets
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/discovery_CV1_clin_u.csv', U[:,0], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/discovery_CV1_rsFC_v.csv', V[:,0], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/testapplied_CV1_clin_u.csv', U_test[:,0], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/testapplied_CV1_rsFC_v.csv', V_test[:,0], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/replicationapplied_CV1_clin_u.csv', U_replication[:,0], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/replicationapplied_CV1_rsFC_v.csv', V_replication[:,0], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/discovery_CV2_clin_u.csv', U[:,1], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/discovery_CV2_rsFC_v.csv', V[:,1], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/testapplied_CV2_clin_u.csv', U_test[:,1], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/testapplied_CV2_rsFC_v.csv', V_test[:,1], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/replicationapplied_CV2_clin_u.csv', U_replication[:,1], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/replicationapplied_CV2_rsFC_v.csv', V_replication[:,1], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/discovery_CV3_clin_u.csv', U[:,2], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/discovery_CV3_rsFC_v.csv', V[:,2], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/testapplied_CV3_clin_u.csv', U_test[:,2], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/testapplied_CV3_rsFC_v.csv', V_test[:,2], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/replicationapplied_CV3_clin_u.csv', U_replication[:,2], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/replicationapplied_CV3_rsFC_v.csv', V_replication[:,2], delimiter=',')


In [56]:
# function to calculate CI for Pearson correlation from:
# https://gist.github.com/zhiyzuo/d38159a7c48b575af3e3de7501462e04
import numpy as np
from scipy import stats

def pearsonr_ci(x,y,alpha=0.05):
    ''' calculate Pearson correlation along with the confidence interval using scipy and numpy
    Parameters
    ----------
    x, y : iterable object such as a list or np.array
      Input for correlation calculation
    alpha : float
      Significance level. 0.05 by default
    Returns
    -------
    r : float
      Pearson's correlation coefficient
    pval : float
      The corresponding p value
    lo, hi : float
      The lower and upper bound of confidence intervals
    '''

    r, p = stats.pearsonr(x,y)
    r_z = np.arctanh(r)
    se = 1/np.sqrt(x.size-3)
    z = stats.norm.ppf(1-alpha/2)
    lo_z, hi_z = r_z-z*se, r_z+z*se
    lo, hi = np.tanh((lo_z, hi_z))
    return r, p, lo, hi

In [60]:
# calculate canonical correlations and CI for each subsample
#CV1
eCCA_canon_corrs1 = np.empty((3,4), dtype=object)
r_d, p_d, lo_d, hi_d = pearsonr_ci(U[:,0],V[:,0],alpha=0.05)
eCCA_canon_corrs1[0,:] = [r_d, p_d, lo_d, hi_d]
r_t, p_t, lo_t, hi_t = pearsonr_ci(U_test[:,0],V_test[:,0],alpha=0.05)
eCCA_canon_corrs1[1,:] = [r_t, p_t, lo_t, hi_t]
r_r, p_r, lo_r, hi_r = pearsonr_ci(U_replication[:,0],V_replication[:,0],alpha=0.05)
eCCA_canon_corrs1[2,:] = [r_r, p_r, lo_r, hi_r]

print(eCCA_canon_corrs1)
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/allsubsets_canoncorrs1.csv', eCCA_canon_corrs1, delimiter=',')

#CV2
eCCA_canon_corrs2 = np.empty((3,4), dtype=object)
r_d, p_d, lo_d, hi_d = pearsonr_ci(U[:,1],V[:,1],alpha=0.05)
eCCA_canon_corrs2[0,:] = [r_d, p_d, lo_d, hi_d]
r_t, p_t, lo_t, hi_t = pearsonr_ci(U_test[:,1],V_test[:,1],alpha=0.05)
eCCA_canon_corrs2[1,:] = [r_t, p_t, lo_t, hi_t]
r_r, p_r, lo_r, hi_r = pearsonr_ci(U_replication[:,1],V_replication[:,1],alpha=0.05)
eCCA_canon_corrs2[2,:] = [r_r, p_r, lo_r, hi_r]

print(eCCA_canon_corrs2)
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/allsubsets_canoncorrs2.csv', eCCA_canon_corrs2, delimiter=',')

#CV3
eCCA_canon_corrs3 = np.empty((3,4), dtype=object)
r_d, p_d, lo_d, hi_d = pearsonr_ci(U[:,2],V[:,2],alpha=0.05)
eCCA_canon_corrs3[0,:] = [r_d, p_d, lo_d, hi_d]
r_t, p_t, lo_t, hi_t = pearsonr_ci(U_test[:,2],V_test[:,2],alpha=0.05)
eCCA_canon_corrs3[1,:] = [r_t, p_t, lo_t, hi_t]
r_r, p_r, lo_r, hi_r = pearsonr_ci(U_replication[:,2],V_replication[:,2],alpha=0.05)
eCCA_canon_corrs3[2,:] = [r_r, p_r, lo_r, hi_r]

print(eCCA_canon_corrs3)
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/allsubsets_canoncorrs3.csv', eCCA_canon_corrs3, delimiter=',')


[[0.18398392413313613 4.5775895185515535e-37 0.15621826408897313
  0.211459039529216]
 [0.05296231715406499 0.4170191842133072 -0.07497397895197153
  0.1791833047926546]
 [0.032475157672906374 0.026680115496863 0.003756611644267189
  0.0611401790010276]]
[[0.12487057011652791 8.4742362791309e-18 0.09662819870030058
  0.1529120244146015]
 [0.045051227291949617 0.4900442242931696 -0.08285472913652812
  0.17149697822950338]
 [0.012576763437265983 0.39085395632178266 -0.016151120858850586
  0.041283900456020206]]
[[0.137518468362067 2.7634050618163e-21 0.10936109133919948
  0.16545532310831787]
 [-0.0165243700391611 0.8002205040017882 -0.14365217731666532
  0.11113995439898254]
 [-0.015598423247342475 0.2872154837564367 -0.044300625758709924
  0.013129509037940473]]


In [58]:
#Refit elastic net CCA in full replication sample

#load selected feature indices file
#allsubsample_selected_features = np.transpose(stable_features[:,:])
#print(allsubsample_selected_features.shape)

#load selected feature indices file - alternative code block
#alldiscovery_selected_features_file = userhome + r'/Desktop/OCS_bigdata_ML_local/data/allDiscovery_selectedFeatsPearson.csv'
#alldiscovery_selected_features_file = userhome + r'/Desktop/OCS_bigdata_ML_local/data/allDiscovery_selectedFeatsSpearman.csv'
#alldiscovery_selected_features = pd.read_csv(alldiscovery_selected_features_file, header = None)
#alldiscovery_selected_features = alldiscovery_selected_features.to_numpy()
#print(alldiscovery_selected_features.shape)

#allsubsample_rsFC_features = rsFC_features[:, allsubsample_selected_features[:,0] - 1]
#print(alldiscovery_rsFC_features)
#print(allsubsample_rsFC_features.shape)

eCCA_fit_replication = ElasticCCA(c = (optim_c1, optim_c2), l1_ratio = optim_l1_ratio, latent_dims=dims).fit([clin_features_replication, rsFC_features_replication])
print('params: ' + str(optim_c1) + ', ' + str(optim_c2) + ', ' + str(optim_l1_ratio))
allsubsample_scores_replication_refitted = eCCA_fit_replication.score((clin_features_replication, rsFC_features_replication))
print('All replication (refitted) CCA scores: ' + str(allsubsample_scores_replication_refitted))

#calculate transformed canonical variate values
U_replication_refitted,V_replication_refitted = eCCA_fit_replication.transform([clin_features_replication, rsFC_features_replication])



/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 0 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 1 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 2 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 3 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 4 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 5 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaco

params: 0.01, 0.01, 0.5
All replication (refitted) CCA scores: [0.1863045  0.16828864 0.14454764 0.10900275 0.11018055 0.11849582
 0.11601035 0.05762238]


/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 7 not converged. Increase number of iterations.
  warnings.warn(


In [61]:
#alldiscovery_loadings = eCCA_fit.get_loadings((clin_features, alldiscovery_rsFC_features))
allsubsample_loadings_replication = eCCA_fit_replication.get_loadings((clin_features_replication, rsFC_features_replication), normalize=True)
allsubsample_loadings_u_replication = allsubsample_loadings_replication[0]
allsubsample_loadings_v_replication = allsubsample_loadings_replication[1]

#correlate discovery and replication clinical loadings
print(pearsonr(allsubsample_loadings_u[:,0], allsubsample_loadings_u_replication[:,0]))

#correlate discovery and replication rsFC loadings
print(pearsonr(allsubsample_loadings_v[:,0], allsubsample_loadings_v_replication[:,0]))


(-0.3270340359359623, 0.4291292597412211)
(-0.47979762759323974, 1.4924798810048743e-06)


In [62]:
# View clinical loadings
#alldiscovery_loadings = eCCA_fit.get_loadings((clin_features, alldiscovery_rsFC_features))
allsubsample_loadings_replication = eCCA_fit_replication.get_loadings((clin_features_replication, rsFC_features_replication), normalize=True)
allsubsample_loadings_u_replication = allsubsample_loadings_replication[0]
allsubsample_loadings_v_replication = allsubsample_loadings_replication[1]

print(allsubsample_loadings_u_replication.shape)
print(allsubsample_loadings_v_replication.shape)

print('All subsample - clinical loadings CV1: \n' + str(allsubsample_loadings_u_replication[:,0]))



(8, 8)
(91, 8)
All subsample - clinical loadings CV1: 
[42.93993215 19.80742764 19.25366694 17.70371119 26.41263349 23.2412553
 27.4474346  40.5266268 ]


In [63]:
# View rs-FC loadings
allsubsample_loadings_v_CV1_replication = np.argsort(abs(allsubsample_loadings_v_replication[:,0]))
#print(alldiscovery_loadings_v_CV1.shape)
print('CV1')
print(allsubsample_loadings_v_CV1_replication[-20:] + 1)
print(allsubsample_loadings_v_replication[allsubsample_loadings_v_CV1_replication[-20:],0])

allsubsample_loadings_v_CV2_replication = np.argsort(abs(allsubsample_loadings_v_replication[:,1]))
#print(alldiscovery_loadings_v_CV2.shape)
print('CV2')
print(allsubsample_loadings_v_CV2_replication[-20:] + 1)
print(allsubsample_loadings_v_replication[allsubsample_loadings_v_CV2_replication[-20:],0])

allsubsample_loadings_v_CV3_replication = np.argsort(abs(allsubsample_loadings_v_replication[:,2]))
#print(alldiscovery_loadings_v_CV3.shape)
print('CV3')
print(allsubsample_loadings_v_CV3_replication[-20:] + 1)
print(allsubsample_loadings_v_replication[allsubsample_loadings_v_CV3_replication[-20:],0])

#allsubsample_loadings_v_CV4 = np.argsort(abs(allsubsample_loadings_v[:,3]))
#print('CV4')
#print(allsubsample_loadings_v_CV4[-20:] + 1)
#print(allsubsample_loadings_v[allsubsample_loadings_v_CV4[-20:],3])

#allsubsample_loadings_v_CV5 = np.argsort(abs(allsubsample_loadings_v[:,4]))
#allsubsample_loadings_v_CV6 = np.argsort(abs(allsubsample_loadings_v[:,5]))
#allsubsample_loadings_v_CV7 = np.argsort(abs(allsubsample_loadings_v[:,6]))
#allsubsample_loadings_v_CV8 = np.argsort(abs(allsubsample_loadings_v[:,7]))

CV1
[37 21 23  3 71 30 45 60 83 16 39  2 47 85 11 24 79 62 38 54]
[ -8.45753934   8.54810523   8.6395108    8.67107987  -8.93030854
  -9.11902448  -9.20248001   9.44513469   9.79749204  10.23721711
  11.10116212  11.63204194 -13.04993577 -13.09691965  13.68166563
  15.23824092  16.34756895  18.15448053  19.01283444  19.92733419]
CV2
[12  6 30 80 55 35 81 91 14 76 42 13 85 75 66  7  4 67 84 43]
[  5.36664225   8.04582768  -9.11902448   5.49581311  -0.69438637
  -6.59388372  -2.57532197  -1.11300247   2.72884979  -6.43647051
   5.79649311  -5.19765741 -13.09691965   2.1544493    0.0614315
  -7.32640297   0.33154328  -3.02178221  -2.02757044   3.2237319 ]
CV3
[81 17 38 22  3 33 73 79 13 60 87 80 25 85 35 16 82 59 29 71]
[ -2.57532197  -4.89495522  19.01283444   5.91397994   8.67107987
   4.63949533  -8.12739386  16.34756895  -5.19765741   9.44513469
   4.46798626   5.49581311  -6.38002373 -13.09691965  -6.59388372
  10.23721711  -2.63284596   2.73373196  -5.95285463  -8.93030854]


In [64]:
# Save files
#replication refitted model
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/replicationrefit_transformed_clin_U.csv', U, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/replicationrefit_transformed_rsFC_V.csv', V, delimiter=',')


In [65]:
# Save canonical variates for all subsets
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/replicationrefit_CV1_clin_u.csv', U_replication_refitted[:,0], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/replicationrefit_CV1_rsFC_v.csv', V_replication_refitted[:,0], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/replicationrefit_CV2_clin_u.csv', U_replication_refitted[:,1], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/replicationrefit_CV2_rsFC_v.csv', V_replication_refitted[:,1], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/replicationrefit_CV3_clin_u.csv', U_replication_refitted[:,2], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/replicationrefit_CV3_rsFC_v.csv', V_replication_refitted[:,2], delimiter=',')


In [66]:
# Save clinical loadings for replication refitted model
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/allreplication_clinloadings_u_CV1norm.csv', allsubsample_loadings_u_replication[:,0], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/allreplication_clinloadings_u_CV2norm.csv', allsubsample_loadings_u_replication[:,1], delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/network-level/results/relaxedexclusion/allreplication_clinloadings_u_CV3norm.csv', allsubsample_loadings_u_replication[:,2], delimiter=',')
